In [1]:
import sys
sys.path.append("..") # import one subdirectory up in files
#sys.path.append("..") # import one subdirectory up in files`
#os.chdir("../holopy/")

# standard imports
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import matplotlib
import time as time
#import json
import pickle

# 3rd party packages
import qiskit as qk
import qiskit.providers.aer.noise as noise
import tenpy as tp

## custom things
from networks.networks import IsoMPS
from networks.isonetwork import QKParamCircuit
import circuits.basic_circuits as circuits

In [39]:
L=10
shots = 400

theta = np.pi/2 # circuit parametert
params = [qk.circuit.Parameter('theta')]
param_dict = {params[0]:theta}

preg = qk.QuantumRegister(1,'p') # physical qubits
breg = qk.QuantumRegister(1,'b') # bond qubits
circ = qk.QuantumCircuit(preg,breg)
circ.rx(0,preg[0])
circ.rx(0,breg[0])
circ.rxx(params[0],preg[0],breg[0])
pcirc = QKParamCircuit(circ,params)
psi = IsoMPS(preg,breg,[pcirc],L=L,bases=['x']*L)

sim_circ = psi.construct_circuit(param_dict)
simulator = qk.Aer.get_backend('qasm_simulator')

for j in range(10):
    result = qk.execute(sim_circ,
               simulator,
               shots=shots).result()
    counts = result.get_counts()
    ms = np.zeros(L)
    for k in counts.keys():
        m_sample = np.array([np.int(x) for x in k.split(" ")])
        ms += counts[k]/shots * m_sample
    print(ms)

[0.48   0.47   0.515  0.535  0.515  0.525  0.4975 0.465  0.485  0.4575]
[0.5225 0.5    0.5175 0.5525 0.4975 0.5225 0.5575 0.495  0.5275 0.475 ]
[0.5575 0.51   0.475  0.53   0.5275 0.49   0.5275 0.5075 0.4825 0.485 ]
[0.4875 0.55   0.5    0.5    0.48   0.4975 0.5    0.5225 0.5025 0.48  ]
[0.4775 0.4625 0.435  0.4975 0.475  0.5    0.535  0.4825 0.51   0.485 ]
[0.5175 0.485  0.49   0.515  0.5375 0.5325 0.495  0.4575 0.4675 0.44  ]
[0.49   0.49   0.475  0.4575 0.535  0.4925 0.4875 0.4475 0.52   0.515 ]
[0.46   0.51   0.4775 0.495  0.47   0.495  0.505  0.4975 0.4875 0.4875]
[0.49   0.4925 0.4825 0.4825 0.5275 0.49   0.51   0.515  0.4575 0.5375]
[0.485  0.5125 0.47   0.5275 0.495  0.4925 0.51   0.48   0.4975 0.5275]


In [23]:
mz = np.zeros(L)
for k in counts.keys():
    k.split(" ")

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'